In [31]:
import os

import numpy as np
import time
import torch
import pickle as pkl

from torch.nn.modules.loss import KLDivLoss
from torch.optim import SGD
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils import data

import models
import opts
from datasets import BCDatasets, tensor_to_numpy, get_train_transform, get_test_transform, IMG, LABEL
from draw_process import draw_progress
from models import weights_init

DoubleTensor = None
FloatTensor = None
LongTensor = None
ByteTensor = None

from scatnet_python import *

In [36]:
data_path = "/workspace/esc"
dataset_name="esc10"
sr=16000
exclude=[5]
dataset = BCDatasets(data_path, dataset_name, sr, exclude,
                     scattering_time_transform = True)

In [37]:
#(channels, H, signal_length)
print(dataset[0]['img'].shape)
print(dataset[1]['img'].shape)

len(dataset)

(220, 1, 46)
(220, 1, 46)


320

In [27]:
# EnvNet layer shapes
print(((24014-8+1) -8 + 1)/160)
print(((40 - 8 + 1)/3 -1 + 1), ((150 - 13 + 1)/3 - 5 + 1)/3)
print(11*14*50)

150.0
11.0 14.0
7700
